# Stock Trading App Tutorial Part 10-12

## Part 10: [Email and SMS Notifications](https://www.youtube.com/watch?v=AuXenr1QB1g&list=PLvzuUVysUFOuoRna8KhschkVVUo2E2g6G&index=11)
- Email options
    - Set up own private server with Linux packages, host it yourself.
    - Use software service (e.g. SendGrid, Mailgun) that have APIs to send emails, add statistics, prevent spam, e.t.c.
    - Use own Gmail.
- https://realpython.com/python-send-email/
  - Use following code to establish secure connection to email server.
  - Apply to `opening_range_breakout.py`.
  - Add details for message to `config.py`.
- Twilio API for SMS notifications:
    - Add phone number to receive messages.
    - Requires Google Fi to do so (10 digit number).
```python
import smtplib, ssl

# Create a secure SSL context at the sart
context = ssl.create_default_context()

# ... 
messages = []
for symbol in symbols: 
# ...

    if not after_opening_bar_breakout.empty: 
        if symbol not in existing_order_symbols: 
            # ... 
            print(f"Placing order for {symbol} at {limit_price}, closed above {opening_range_high} at {after_opening_bar_breakout.iloc[0]}")
            messages.append(f"Placing order for {symbol} at {limit_price}, closed above {opening_range_high}\n\n{after_opening_bar_breakout.iloc[0]}\n\n")
            api.submit_order(
            # ... 
        else: 
            print(f"Already an order for {symbol}, will skip.")

with smtplib.SMTP_SSL(config.EMAIL_HOST, config.EMAIL_PORT, context=context) as server:
    server.login(config.EMAIL_ADDRESS, config.EMAIL_PASSWORD)
    # TODO: Send email here
    email_message = f"Subject Trade Notifications for {current_date}\n\n"
    email_message += "\n\n".join(messages)
    server.sendmail(config.EMAIL_ADDRESS, config.EMAIL_ADDRESS) # Send email from self to self
    server.sendmail(config.EMAIL_ADDRESS, config.EMAIL_SMS) # Send SMS
```

## Part 11: [Tulip Indicators](https://www.youtube.com/watch?v=ryqsCbVSHtY&list=PLvzuUVysUFOuoRna8KhschkVVUo2E2g6G&index=12)
- [Tulip Indicators](https://tulipindicators.org): Technical analysis indicator library for financial analysis. 
- Contains 104 indicators.
- Download with `pip install newtulipy`.
    - Requires `pip install Cython`, `pip install --upgrade pip setuptools wheel`.
    - Use along with `numpy`.
    - CANNOT BE USED, PLEASE USE TA-lib INSTEAD (I HAVE TRIED FOR HOURS).
- Download with `pip install ta-lib`
    - NOTE TO SELF: This got downloaded to `anaconda` instead, use `/opt/anaconda3/bin/jupyter_mac.command ; exit;` to run Jupyter.
- Want to filter from stock list; could dynamically pull price data from database, then apply indicator, filter and return results, but would be inefficient.
    - Will calculate and store in database within `populate_prices.py` as data comes in, since data will not change unless it is real time data.
    - Use indicators as filter to come up with trade ideas; select stocks with bullish/bearish bias.
    - Pass `numpy` array of prices, then call indicator of interest.
    - Create new column within `stock_prices` table for each new indicator (Can be null).
- `populate_prices.py`
```python
# ...
current_date = dt.datetime.strptime("2022-06-21", '%Y-%m-%d').date()
for i in range(0, len(symbols), chunk_size):
    symbol_chunk = symbols[i:i+chunk_size]

    #  Need to acquire list of most recent closes for SMA 
    # barsets = api.get_bars(symbol_chunk, TimeFrame.Day, start = oneDayAgo, end = oneDayAgo, limit = 1)._raw # Results not guranteed
    barsets = api.get_bars(symbol_chunk,TimeFrame.Day,"2022-03-21", "2022-06-21")._raw
    barsets_final = collections.defaultdict(list) # Obtain barsets split by symbol
    for bar in barsets:
        barsets_final[bar['S']].append(bar)
    # print(barsets[:10])
    # print(barsets_final)
    # print(list(barsets_final.keys()))

    for symbol in list(barsets_final.keys()): # For each symbol
        recent_closes = [bar["c"] for bar in barsets_final[symbol]]
        for bar in barsets_final[symbol]: # Loop through each bar in barset of given symbol
            print(f"processing symbol {symbol}")
            # print(bar)
            stock_id = stock_dict[bar["S"]]
            stock_time = dt.datetime.strptime(bar["t"], '%Y-%m-%dT%H:%M:%SZ').date() # MODIFICATION MADE TO MAKE PROCESS EASIER
            if len(recent_closes) >= 50 and current_date == stock_time:
                sma_20 = talib.SMA(numpy.array(recent_closes, dtype=numpy.float64), 20)[-1]
                sma_50 = talib.SMA(numpy.array(recent_closes, dtype=numpy.float64), 50)[-1]
                rsi_14 = talib.RSI(numpy.array(recent_closes, dtype=numpy.float64), 14)[-1]
            else: 
                sma_20, sma_50, rsi_14 = None, None, None
            
            cursor.execute("""
            INSERT INTO stock_price (stock_id, date, open, high, low, close, volume, sma_20, sma_50, rsi_14)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (stock_id, stock_time, bar["o"], bar["h"], bar["l"], bar["c"], bar["v"], sma_20, sma_50, rsi_14))  
connection.commit() # To commit changes to databases
```
- `main.py`
```python
@app.get("/")
def index(request: Request): 
    # ... 
    # current_date = date.today().isoformat()
    current_date = "2022-06-21"
    
    cursor.execute("""
        SELECT symbol, sma_20, sma_50, rsi_14, close 
        FROM stock JOIN stock_price ON stock_price.stock_id = stock.id
        WHERE date = ?
    """, (current_date,))
    indicator_rows = cursor.fetchall()
    indicator_values = {}
    for row in indicator_rows: 
        indicator_values[row["symbol"]] = row
    
    return templates.TemplateResponse("index.html", context={"request": request, "stocks": rows, "indicator_values": indicator_values})
```

## Part 12: [UI Bug Fixes / Daylight Savings](https://www.youtube.com/watch?v=AlgFvJf6Vxo&list=PLvzuUVysUFOuoRna8KhschkVVUo2E2g6G&index=13)


- Error: If system run on weekend, prices may not be available and thus sets will be empty.
    - Modify such that most recent price is recieved instead; use `WHERE date = (SELECT max(date) FROM stock_price)` for `cursor.execute()` of `rows` and `indicator_rows`.
    - `indicator_rows`:
```python
cursor.execute("""
    SELECT symbol, sma_20, sma_50, rsi_14, close 
    FROM stock JOIN stock_price ON stock_price.stock_id = stock.id
    WHERE date = (SELECT max(date) FROM stock_price)
""")
indicator_rows = cursor.fetchall()
indicator_values = {}
for row in indicator_rows: 
    indicator_values[row["symbol"]] = row
```
- Selection box within `index.html` automatically defaults to "All Stocks" when changing pages, want to stick to current option.
```html
<h1 class="ui center aligned header">Stock List</h1>
<form method="get">
    <select name="filter">
        <option value="">All Stocks</option>
        <option 
        {% if request.query_params.filter == "new_intraday_highs" %} selected="selected" {% endif %} 
        value="new_intraday_highs">New Intraday Highs</option>
        <option 
        {% if request.query_params.filter == "new_closing_highs" %} selected="selected" {% endif %} 
        value="new_closing_highs">New Closing Highs</option>
        <option 
        {% if request.query_params.filter == "new_intraday_lows" %} selected="selected" {% endif %} 
        value="new_intraday_lows">New Intraday Lows</option>
        <option 
        {% if request.query_params.filter == "new_closing_lows" %} selected="selected" {% endif %} 
        value="new_closing_lows">New Closing Lows</option>
    </select>
    <input type="submit"/>
</form>
```